# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902020


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:38,  3.51s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:08,  2.52s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:45,  1.82s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:33,  1.39s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:01,  3.88it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  4.43it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  4.34it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.88it/s]

Jes8x                                 0.001427
CTidwell3                             0.000598
JessieDawn                            0.053820
equidad1                              0.068043
Drew_Farnsworth                       0.000245
not-logged-in-621b62a5f3840545c538    0.004896
Thedunkmasta                          0.000991
not-logged-in-0997cfcba8da23453381    0.000452
mlang2017                             0.017828
DrJackie                              0.001653
jstucky                               0.000439
capraj1                               0.001386
jedkat                                0.016810
not-logged-in-9a45911eb71184fe7c7f    0.002185
not-logged-in-cad91d8dcf85ed0434df    0.003947
quirito19                             0.002499
pangeli5                              0.001192
TRISTANM18                            0.001467
Jatkins9752                           0.000433
Lavadude                              0.006339
not-logged-in-ab5f58ee1e61681e9014    0.423210
atoderov     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())